In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import pickle

In [ ]:
import tqdm
import pandas as pd
import selfies as sf
from rdkit import Chem

In [ ]:
REBADD_LIB_PATH = os.path.abspath(os.pardir)
if REBADD_LIB_PATH not in sys.path:
    sys.path = [REBADD_LIB_PATH] + sys.path
    
from rebadd.datautils import get_fragment_from_selfies

In [ ]:
class DATACONFIGS:
    def __init__(self):
        ## input
        self.input_dir = os.path.join(os.pardir, 'data', 'chembl')
        self.train_data_path = os.path.join(self.input_dir, 'chembl_train_full.csv')
        ## output
        self.output_dir = os.path.join('processed_data', 'gsk3_jnk3_qed_sa')
        assert os.path.exists(self.output_dir)

dataconfigs = DATACONFIGS()

In [ ]:
df_train = pd.read_csv(dataconfigs.train_data_path)

df_train

In [ ]:
df_trainable = df_train[ (df_train['gsk3'] > 0.5) | (df_train['jnk3'] > 0.5) | (df_train['qed'] > 0.6) | (df_train['sa'] < 4.001)]
df_trainable = df_trainable[ (df_trainable['gsk3'] > 0.01) | (df_trainable['jnk3'] > 0.01) ]
df_trainable = df_trainable.reset_index(drop=True)

df_trainable

In [ ]:
data = df_trainable.loc[:,'smiles'].values.tolist()

with open(os.path.join(dataconfigs.output_dir, 'trainable_smiles.txt'), 'w') as fout:
    for smi in data:
        fout.write(f'{smi}\n')

In [ ]:
print(f'Number of training data (raw): {len(data)}')
data[:5]

In [ ]:
def make_selfies_data(smiles_iter):
    selfies_list = []
    fragments_list = []
    
    for smi in tqdm.tqdm(smiles_iter):
        
        try:
            mol = Chem.MolFromSmiles(smi)
            Chem.Kekulize(mol)
        except:
            continue

        try:
            smi = Chem.MolToSmiles(mol, canonical=True, doRandom=False, isomericSmiles=False, kekuleSmiles=True)
            sel = sf.encoder(smi)
            frags = get_fragment_from_selfies([sel], use_tqdm=False)[0]
            if max([sf.len_selfies(x) for x in frags]) < 16:
                selfies_list.append(sel)
                fragments_list.append(frags)
        except AssertionError:
            pass
        except sf.EncoderError:
            pass
    
    return selfies_list, fragments_list

In [ ]:
selfies_list, fragments_list = make_selfies_data(data)

In [ ]:
print(f'Number of training data (selfies): {len(selfies_list)}')
selfies_list[:5]

In [ ]:
print(f'maxlen of fragments: {max([len(fragments) for fragments in fragments_list])}')
fragments_list[:2]

In [ ]:
with open(os.path.join(dataconfigs.output_dir, 'fragments_list.pkl'), 'wb') as fout:
    pickle.dump(fragments_list, fout)

In [ ]:
vocabs = set()
for fragments in tqdm.tqdm(fragments_list):
    vocabs = vocabs.union(set(fragments))

vocabs = sorted(vocabs)

In [ ]:
print(f'Number of vocabulary(unique fragments): {len(vocabs)}')
vocabs

In [ ]:
with open(os.path.join(dataconfigs.output_dir, 'vocabulary.csv'), 'w') as fout:
    for v in vocabs:
        fout.write(f'{v}\n')

In [ ]:
with open(os.path.join(dataconfigs.output_dir, 'selfies.csv'), 'w') as fout:
    for selfies in selfies_list:
        fout.write(f'{selfies}\n')